This is my attempt to explain `X`, `Y`, `Dir` and `Orientation` as I understand them.

To start, **we'll always want to assume we are observing the field from the home team's sideline.** The [image at the bottom of the data section](https://www.kaggle.com/c/nfl-big-data-bowl-2020/data) is a good one to use as long as you don't look at the "Increasing Dir and O" bit which is not quite right.

![Imgur](https://i.imgur.com/I0iPp0A.png)

A few notes before moving on

1. X and Y give the player positions on the field
2. The field is 120 yards long by 53.3 yards wide but players can go out of bounds so don't expect every player's (X,Y) to be within that range
3. Suppose a rusher's X coordinate is 20. If *PlayDirection* is 'left', he is 10 yards from scoring. If *PlayDirection* is 'right', he is 90 yards from scoring.

# The Two Coordinate Reference Systems
*Dir* and *Orientation* are both given as angles in the range \[0, 360\]. *Dir* gives the angle the player is moving and *Orientation* gives the angle the player is facing, each at the time the ball is handed to the rusher. Through some experimentation, I've determined that there are two different coordinate reference systems for these measuring angles. They are

## 1. (0° on right, clockwise rotation)
**Used for:**
1. *Orientation* where *Season* == 2017
![Imgur](https://i.imgur.com/MFAW8FQ.png)

## 2. (0° on top, clockwise rotation)
1. *Orientation* where *Season* > 2017 and
2. *Dir*
![Imgur](https://i.imgur.com/a0jxkYp.png)

(Thanks to Peter Hurford for [pointing out that Orientation is shifted by 90 degrees in 2017](https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/113384#latest-653467))

# How to Standardize?
There are a number of ways you could standardize this data. I prefer to transform every angle into the system (0° on right, counter clockwise rotation), because this makes cos(angle) increase in the same direction as X and sin(angle) increase in the same direction as Y. I've done this transformation in the `plot_play()` method below.

# Examples
Here's a dirty `plot_play()` method with some examples including links to youtube videos with the corresponding play.

In [ ]:
import numpy as np
import pandas as pd
import getpass
import matplotlib.pyplot as plt

# ==== Setup =======================================================================================

np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 0)

if getpass.getuser() in ['Ben']:
    input_file = 'input/train.csv'
else:
    from kaggle.competitions import nflrush
    env = nflrush.make_env()
    input_file = '/kaggle/input/nfl-big-data-bowl-2020/train.csv'

# read train
train = pd.read_csv(filepath_or_buffer = input_file, low_memory = False)

In [ ]:
def plot_play(playId, arrows_as = 'Dir'):
    """
    Plot a single play
    :param playId: id of the play 
    :param arrows_as: should be either 'Dir' or 'Orientation'
    :return: plot of players with arrows
    """

    # Get 'play players' and 'play' from train
    play_players = train.loc[train.PlayId == playId].copy()
    play = play_players.iloc[[0]]

    # Determine which players are on offense by identifying which team the rusher is on
    rusher = play_players.loc[play_players.NflId == play_players.NflIdRusher]
    play_players.loc[:, 'PlayerOnOffense'] = play_players.Team.values == rusher.Team.values

    # Create field 'ArrowAngle'
    play_players['ArrowAngle'] = play_players['Dir'] if arrows_as == 'Dir' else play_players['Orientation']
    if (play.Season.values[0] == 2017 and arrows_as == 'Orientation'):
        play_players['ArrowAngle'] = (360 - play_players.ArrowAngle).mod(360)
    else:
        play_players['ArrowAngle'] = (360 - play_players.ArrowAngle + 90).mod(360)

    # Create fields Arrow_dx, Arrow_dy
    play_players['Arrow_dx'] = np.cos(play_players.ArrowAngle * (np.pi/180))
    play_players['Arrow_dy'] = np.sin(play_players.ArrowAngle * (np.pi/180))

    # Split offense and defense players
    play_players_offense = play_players.loc[play_players.PlayerOnOffense].copy()
    play_players_defense = play_players.loc[~play_players.PlayerOnOffense].copy()

    # Plot
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.axvline(x=10, linewidth=1)
    ax.axvline(x=110, linewidth=1)
    ax.axhline(y=0, linewidth=1)
    ax.axhline(y=53.5, linewidth=1)
    ax.scatter(play_players_offense.X, play_players_offense.Y, color="red", label="offense", s = 20)
    ax.scatter(play_players_defense.X, play_players_defense.Y, color="blue", label="defense", s = 20)
    for i in range(0, play_players.shape[0]):
        ax.arrow(
            x = play_players.X.values[i],
            y = play_players.Y.values[i],
            dx = play_players.Arrow_dx.values[i],
            dy = play_players.Arrow_dy.values[i],
            head_width=0.5,
            head_length=0.5,
            fc='k',
            ec='k'
        )
    ax.text(60, 0 - 5, 'Home Sideline', horizontalalignment='center', verticalalignment='bottom')
    ax.text(60, 53.5 + 5, 'Away Sideline', horizontalalignment='center', verticalalignment='top', rotation=180)
    ax.set_xlim(0, 120)
    ax.set_ylim(0 - 5, 53.5 + 5)
    ax.set_title(f"PlayId: {play.PlayId.values[0]} (moving {play.PlayDirection.values[0]})\n{arrows_as}")
    fig.legend()
    fig.show()

In [ ]:
# Derrick Henry, 2018, moving right, https://www.youtube.com/watch?v=tlZvgdcIXvI
plot_play(20181206001238, 'Orientation')

In [ ]:
# Lamar Miller, 2018, moving left, https://www.youtube.com/watch?v=p-ptA3nQxCA
plot_play(20181126001222, 'Orientation')

In [ ]:
# Nick Chubb, 2018, moving left, https://www.youtube.com/watch?v=NvQiykZIBNA
plot_play(20181111022155, 'Orientation')

In [ ]:
# Adrian Peterson, 2018, moving left, https://www.youtube.com/watch?v=AMLKvNs2Ec8
plot_play(20181203001224, 'Orientation')

In [ ]:
# Leonard Fournette, 2017, moving right, https://youtu.be/Dp3zkB3NRDA?t=114
plot_play(20171008074020, 'Orientation')

In [ ]:
# Melvin Gordon, 2017, moving right, https://www.youtube.com/watch?v=oUHaQKmyn7U
plot_play(20171029030426, 'Orientation')

In [ ]:
# Bilal Powell, 2017, moving left, https://www.youtube.com/watch?v=zDtDanILhAc
plot_play(20171001080397, 'Orientation')

In [ ]:
# Saquon Barkley, 2018, moving left, https://www.youtube.com/watch?v=E4IesbDwpq4
plot_play(20181209081494, 'Orientation')

In [ ]:
# Kerryon Johnson, 2018, moving right, https://youtu.be/cgZnUFAtd0c?t=27
plot_play(20181021060782, 'Orientation')

In [ ]:
# Alvin Kamara, 2017, moving right, https://www.youtube.com/watch?v=4XAYJKiT2rc
plot_play(20171126070740, 'Orientation')